In [1]:
# Importamos las librerías necesarias
import pandas as pd
from datetime import datetime
import geopandas as gpd

In [2]:
# Función para calcular la edad a partir de la fecha de nacimiento
def calcular_edad(fecha_nacimiento):
    hoy = datetime.today()
    return hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))

# Cargar la base de datos de usuarios
usuarios = pd.read_excel(r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\EMA\ema 2023.xlsx", 'usuarios_2023')

# Convertir la columna 'Birthdate' a formato de fecha
usuarios['Birthdate'] = pd.to_datetime(usuarios['Birthdate'], format='%d/%m/%Y')

# Calcular la edad y almacenarla en una nueva columna 'Edad'
usuarios['Edad'] = usuarios['Birthdate'].apply(calcular_edad)

# Agrupar por 'Shipping Zip/postal Code' y contar cada género, además de agregar la población total
usuarios_agrupados = usuarios.groupby('Shipping Zip/postal Code').agg(
    TOTAL=('Gender', 'size'),  
    MALE=('Gender', lambda x: (x == 'MALE').sum()),
    FEMALE=('Gender', lambda x: (x == 'FEMALE').sum()),
    OTHER=('Gender', lambda x: (x == 'OTHER').sum())
).reset_index()

# Crear categorías quinquenales de edades y contarlas
usuarios['Edad_Grupo'] = pd.cut(usuarios['Edad'], bins=[15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 100],
                                labels=['16 a 20', '21 a 25', '26 a 30', '31 a 35', '36 a 40', '41 a 45', 
                                        '46 a 50', '51 a 55', '56 a 60', '61 a 65', '66 a 70', '71 a 100'])

# Contar las edades quinquenales por código postal
edad_quinquenal = usuarios.pivot_table(index='Shipping Zip/postal Code', columns='Edad_Grupo', 
                                       aggfunc='size', fill_value=0)

# Cargar el archivo de encuestas
encuestas = pd.read_excel(r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\EMA\Encuesta_Muestra_Representativa_v1_240627.xlsx")

# Contar los valores únicos de la columna de ingreso mensual por CP
ingreso_mensual = encuestas.pivot_table(index='Codigo_Postal', columns='¿Cuál es tu ingreso mensual aproximado? (personal)Pesos mexicanos', 
                                        aggfunc='size', fill_value=0)

# Excluir la columna "Prefiero no responder" y calcular porcentajes
if 'Prefiero no responder' in ingreso_mensual.columns:
    validas = ingreso_mensual.drop(columns=['Prefiero no responder'])
else:
    validas = ingreso_mensual.copy()

suma_validas = validas.sum(axis=1)
ingreso_porcentajes = validas.div(suma_validas, axis=0) * 100

# Contar los valores únicos de la columna de ingreso mensual por CP
grado_estudios = encuestas.pivot_table(index='Codigo_Postal', columns='Máximo grado de estudios', 
                                        aggfunc='size', fill_value=0)

# Excluir la columna "Prefiero no responder" y calcular porcentajes
if 'Prefiero no responder' in grado_estudios.columns:
    validas = grado_estudios.drop(columns=['Prefiero no responder'])
else:
    validas = grado_estudios.copy()

suma_validas = validas.sum(axis=1)
estudios_porc = validas.div(suma_validas, axis=0) * 100

# Unir todos los resultados en un solo DataFrame final
resultado_final = usuarios_agrupados.join(edad_quinquenal, on='Shipping Zip/postal Code', how='left')
resultado_final = resultado_final.join(ingreso_mensual, on='Shipping Zip/postal Code', how='left')
resultado_final = resultado_final.join(ingreso_porcentajes.add_suffix('_%'), on='Shipping Zip/postal Code', how='left')
resultado_final = resultado_final.join(grado_estudios, on='Shipping Zip/postal Code', how='left')
resultado_final = resultado_final.join(estudios_porc.add_suffix('_%'), on='Shipping Zip/postal Code', how='left')

# Guardar el resultado en un archivo Excel
resultado_final.to_excel(r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\EMA\Usuarios_CP.xlsx", index=False)

print("Análisis completado y guardado en 'Usuarios_CP.xlsx'")

Análisis completado y guardado en 'Usuarios_CP.xlsx'


In [7]:
# Unir a shape CP

# Cargar el archivo shapefile de los códigos postales
shapefile = gpd.read_file(r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Códigos Postales JALISCO\CP_JAL.shp")

# Cargar el archivo Excel con los resultados del análisis
resultado = pd.read_excel(r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\EMA\Usuarios_CP.xlsx")

# Convertir las columnas de código postal a string para hacer la unión
shapefile['C__digo_Po'] = shapefile['C__digo_Po'].astype(str)
resultado['Shipping Zip/postal Code'] = resultado['Shipping Zip/postal Code'].astype(str)

# Unir los datos del Excel al shapefile usando el campo del código postal (asegúrate que tienen el mismo nombre)
shapefile_unido = shapefile.merge(resultado, left_on='C__digo_Po', right_on='Shipping Zip/postal Code')

# Conservar sólo las columnas de interés, más la columna de geometría
columnas_interes = ['C__digo_Po'] + list(resultado.columns) + ['geometry']
shapefile_final = shapefile_unido[columnas_interes]

# Guardar el resultado en un nuevo archivo shapefile
shapefile_final.to_file(r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\EMA\Usuarios_CP.gpkg",layer='2023_MiBici_UsEc')
                        
print("Unión completada y guardada en 'Usuarios_CP.gpkg'")


Unión completada y guardada en 'Usuarios_CP.gpkg'
